# Assignment 3 - DATA.ML.360

In [3]:
%pip install pandas
%pip install surprise

import pandas as pd

You should consider upgrading via the '/Users/laurira/uni/recsys/assignment3/a3_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/laurira/uni/recsys/assignment3/a3_venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Read the data file and see how it looks like
df = pd.read_csv('u.data', sep='\t', header=None)

# Add column names and check few rows of the dataset
df.columns = ["user_id", "movie_id", "rating", "timestamp"]
df = df.drop("timestamp", axis=1)

df.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


### Singular Value Decomposition (SVD)
Matrix factorization
Explanation here, what and why

Steps:
- Train the model sequentially
- Create group recommendations for that sequence
- Lump the sequences recommendations together, get top 10 recommendations

Sources used:

https://gregorygundersen.com/blog/2018/12/10/svd/#:~:text=The%20singular%20values%20referred%20to,our%20transformation%20flattens%20our%20square.

https://towardsdatascience.com/how-to-build-a-movie-recommendation-system-67e321339109

https://www.math3ma.com/blog/understanding-entanglement-with-svd


In [132]:
from surprise import Dataset
from surprise import Reader
from surprise import SVD

def create_group_recommendations(users):
    # Create a dataset
    reader = Reader(rating_scale=(1, 5))

    # Save recommendations from each sequence here
    group_recommendations = []

    # Loop through sequences
    for round in range(3):
        # Train the dataset for each round
        data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)
        trainset = data.build_full_trainset()

        # Intialize the svd algorithm and train the dataset
        svd = SVD()
        svd.fit(trainset)

        # Go thorugh each user and create predictions.
        for user_id in users:
            # Get movies that user hasnt rated yet
            user_ratings = df[df['user_id'] == user_id]
            user_unrated_movies = df[~df['movie_id'].isin(user_ratings['movie_id'])]['movie_id'].unique()

            user_recommendations = []

            # Loop through the unrated movies
            for movie_id in user_unrated_movies:
                # Create a predicted score for the movie
                prediction = svd.predict(user_id, movie_id)
                # Add it to the recommendations
                user_recommendations.append((user_id, movie_id, prediction.est))

            # Retrun the top 10 best rated predictions
            user_recommendations = sorted(user_recommendations, key=lambda x: x[2], reverse=True)[:10]
            group_recommendations.extend(user_recommendations)

    return group_recommendations

user_list = [1, 2, 3, 5, 6, 7, 8, 9, 10]
result = create_group_recommendations(user_list)

# thing = pd.DataFrame(result, columns=["user_id", "movie_id", "predicted_rating"])
# thing = thing.groupby('movie_id')['predicted_rating'].mean()
# thing = thing.sort_values(ascending=False)[:10]
# display(thing)


### Average Weighted ratings

Modified method for sequential recommendations

In [133]:
# Select a set of users and create recommendations for them
user_list = [1, 2, 3, 5, 6, 7, 8, 9, 10]
result = create_group_recommendations(user_list)

# Create a dataframe from the results
group_df = pd.DataFrame(result, columns=["user_id", "movie_id", "predicted_rating"])

# Create a dataframe for user weights &
# calculate the weight for each user from their average predicted rating
user_weights = group_df.groupby('user_id')['predicted_rating'].mean().reset_index()
user_weights.columns = ['user_id', 'weight']

# Merge weights with the original group recommendations
group_df = pd.merge(group_df, user_weights, on='user_id', how='left')

# Multioply the original prediction with the user weight -> weighted rating for each item
group_df['weighted_rating'] = group_df['predicted_rating'] * group_df['weight']

# Divide total weighted ratings by total weights -> average rating for each tiem
weighted_ratings = group_df.groupby('movie_id')['weighted_rating'].sum() / group_df.groupby('movie_id')['weight'].sum()

# Get top 10 recommendations based on the weighted average ratings
top_recommendations = weighted_ratings.sort_values(ascending=False)[:10]
display(top_recommendations)

movie_id
59      4.997454
170     4.942098
22      4.941217
173     4.920384
427     4.852079
302     4.845650
169     4.844958
1142    4.839546
318     4.838508
272     4.804268
dtype: float64